<a href="https://colab.research.google.com/github/dejiandrew/nba-award-predictor/blob/deji/lightgbm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm *.csv
!rm *.pkl

rm: cannot remove '*.pkl': No such file or directory


In [ ]:
!gsutil -m cp -r gs://nba_award_predictor/nba_data/models/ag_models ./

Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F6/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/learner.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/metadata.json...
Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F7/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F4/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F1/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F5/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F8/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F2/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F3/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/a

In [ ]:
!pip install wget
!pip install autogluon
!pip install lightgbm
import pandas as pd
import wget
import duckdb
import lightgbm as lgb
import joblib
import numpy as np

In [ ]:
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/features-overall-weekly.csv')
features_overall_weekly_df = pd.read_csv('features-overall-weekly.csv')

In [ ]:
# Remove unwanted features
features_overall_weekly_df = features_overall_weekly_df.drop(columns=['league_pts_mean', 'league_pts_std',
       'league_ast_mean', 'league_ast_std', 'league_pm_mean', 'league_pm_std',
       'z_s_pts', 'z_s_ast', 'z_s_pm'])
features_overall_weekly_df

,player_id,full_name,team,season,week,week_start,conference,pow_conference,games_played_this_week,numMinutes,...,points_mean_season,points_std_season,assists_mean_season,assists_std_season,plusMinusPoints_mean_season,plusMinusPoints_std_season,z_pts,z_ast,z_pm,breakout_score
0,305,Robert Parish,Warriors,1979,1,1979-12-31,West,West,3,101.00,...,63.000000,7.874008,8.600000,3.382307,0.000000,0.000000,-0.254000,-1.655675,0.000000,-0.623703
1,76003,Kareem Abdul-Jabbar,Lakers,1979,1,1979-12-31,West,West,2,82.00,...,105.000000,25.674890,19.400000,7.200000,0.000000,0.000000,-1.986377,-0.750000,0.000000,-1.218188
2,76005,Tom Abernethy,Warriors,1979,1,1979-12-31,West,West,1,11.00,...,22.000000,13.190906,4.750000,2.680951,0.000000,0.000000,-1.667816,-1.771759,0.000000,-1.365436
3,76011,Alvan Adams,Suns,1979,1,1979-12-31,West,West,2,64.00,...,44.200000,27.095387,13.800000,7.884161,0.000000,0.000000,-0.634794,-0.862489,0.000000,-0.576144
4,76085,James Bailey,SuperSonics,1979,1,1979-12-31,West,West,4,41.00,...,18.000000,6.745369,0.750000,0.433013,0.000000,0.000000,-0.444750,2.886751,0.000000,0.643651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235264,1642942,Jahmai Mashack,Grizzlies,2025,51,2025-12-15,West,West,2,9.48,...,0.000000,0.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
235265,1642948,Ryan Nembhard,Mavericks,2025,51,2025-12-15,West,West,3,85.60,...,32.333333,23.976841,17.333333,12.472191,-3.333333,11.145502,-0.222437,0.454344,-0.239259,-0.022767
235266,1642949,Yanic Konan Niederhauser,Clippers,2025,51,2025-12-15,West,West,1,10.20,...,12.600000,10.800000,1.800000,2.400000,-12.000000,28.586710,-0.611111,-0.750000,0.944495,-0.341657
235267,1642954,Will Richard,Warriors,2025,51,2025-12-15,West,West,1,19.15,...,27.500000,15.392098,3.166667,1.572330,6.000000,25.252063,-0.487263,-2.013996,-0.198004,-0.887431


In [ ]:
features_overall_weekly_df.columns

Index(['player_id', 'full_name', 'team', 'season', 'week', 'week_start',
       'conference', 'pow_conference', 'games_played_this_week', 'numMinutes',
       'points', 'assists', 'blocks', 'steals', 'reboundsTotal',
       'reboundsDefensive', 'reboundsOffensive', 'fieldGoalsAttempted',
       'fieldGoalsMade', 'threePointersAttempted', 'threePointersMade',
       'freeThrowsAttempted', 'freeThrowsMade', 'turnovers', 'foulsPersonal',
       'plusMinusPoints', 'wins_this_week', 'wins_vs_team_with_all_nba_player',
       'is_win_vs_over_500', 'opponent_has_all_nba', 'avg_opp_score',
       'avg_opp_winrate_prior', 'avg_opp_wins_prior', 'avg_opp_losses_prior',
       'away_games_prior', 'away_losses_prior', 'away_win_streak_prior',
       'away_wins_prior', 'home_games_prior', 'home_losses_prior',
       'home_win_streak_prior', 'home_wins_prior', 'losses_prior',
       'wins_vs_over_500_prior', 'won_player_of_the_week',
       'all_star_this_season', 'mvp_this_season',
       'all_nba_f

In [ ]:
query = """
WITH CTE AS (
SELECT team, season, week, SUM(assists) AS team_assists
FROM features_overall_weekly_df
GROUP BY team, season, week
)
,CTE2 AS (

SELECT team, season, week, SUM(fieldGoalsMade) AS team_fg
FROM features_overall_weekly_df
GROUP BY team, season, week
)
, CTE3 AS (
SELECT CTE.team, CTE.season, CTE.week, team_assists, team_fg
FROM CTE
JOIN CTE2
ON CTE.team = CTE2.team AND CTE.season = CTE2.season AND CTE.week = CTE2.week
)
, CTE4 AS (

SELECT team, season, week, SUM(points)/team_gms AS team_ppg
FROM features_overall_weekly_df
GROUP BY team, season, week, team_gms
)

, CTE5 AS (
SELECT *
,team_pts/team_gms AS team_ppg
FROM features_overall_weekly_df a
JOIN CTE3 b
ON a.team=b.team AND a.season = b.season AND a.week=b.week
)

, CTE6 AS (
  SELECT *,
  CASE WHEN team_ppg > 186 THEN 115 ELSE team_ppg END AS team_ppg_fixed  -- If team_ppg shows as greater than 186, set it to 115
  FROM CTE5
)

SELECT *,
AVG(team_ppg_fixed) OVER(PARTITION BY season, week) AS lg_ppg
FROM CTE6

"""
features_overall_weekly_df = duckdb.query(query).df().drop(columns=['team_1','season_1','week_1','team_ppg'])
features_overall_weekly_df

# League-wide statistics (scalars)
lg_AST = features_overall_weekly_df['assists'].mean()
lg_FG = features_overall_weekly_df['fieldGoalsMade'].mean()
lg_FT = features_overall_weekly_df['freeThrowsMade'].mean()
lg_PF = features_overall_weekly_df['foulsPersonal'].mean()
lg_FTA = features_overall_weekly_df['freeThrowsAttempted'].mean()

factor = (2/3) - (0.5 * (lg_AST / lg_FG)) / (2 * (lg_FG / lg_FT))
vop = features_overall_weekly_df['points'].mean() / (features_overall_weekly_df['fieldGoalsAttempted'].mean() - features_overall_weekly_df['reboundsOffensive'].mean() + features_overall_weekly_df['turnovers'].mean() + 0.44 * lg_FTA)
drb_pct = (features_overall_weekly_df['reboundsTotal'].mean() - features_overall_weekly_df['reboundsOffensive'].mean()) / features_overall_weekly_df['reboundsTotal'].mean()

features_overall_weekly_df['u_per'] = (1 / features_overall_weekly_df['numMinutes']) * (
    features_overall_weekly_df['threePointersMade']
    + (2/3) * features_overall_weekly_df['assists']
    + (2 - factor * (features_overall_weekly_df['team_assists'] / features_overall_weekly_df['team_fg'])) * features_overall_weekly_df['fieldGoalsMade']
    + (features_overall_weekly_df['freeThrowsMade'] * 0.5 * (1 + (1 - (features_overall_weekly_df['team_assists'] / features_overall_weekly_df['team_fg'])) + (2/3) * (features_overall_weekly_df['team_assists'] / features_overall_weekly_df['team_fg'])))
    - vop * features_overall_weekly_df['turnovers']
    - vop * drb_pct * (features_overall_weekly_df['fieldGoalsAttempted'] - features_overall_weekly_df['fieldGoalsMade'])
    - vop * 0.44 * (0.44 + (0.56 * drb_pct)) * (features_overall_weekly_df['freeThrowsAttempted'] - features_overall_weekly_df['freeThrowsMade'])
    + vop * (1 - drb_pct) * (features_overall_weekly_df['reboundsTotal'] - features_overall_weekly_df['reboundsOffensive'])
    + vop * drb_pct * features_overall_weekly_df['reboundsOffensive']
    + vop * features_overall_weekly_df['steals']
    + vop * drb_pct * features_overall_weekly_df['blocks']
    - features_overall_weekly_df['foulsPersonal'] * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * vop)
)

# First, check for problematic values
print("Minutes == 0:", (features_overall_weekly_df['numMinutes'] == 0).sum())
print("team_fg == 0:", (features_overall_weekly_df['team_fg'] == 0).sum())
print("NaN values in key columns:")
print(features_overall_weekly_df[['numMinutes', 'team_assists', 'team_fg']].isna().sum())

# Filter to only calculate for valid rows
valid_mask = (
    (features_overall_weekly_df['numMinutes'] > 0) &
    (features_overall_weekly_df['team_fg'] > 0) &
    features_overall_weekly_df['numMinutes'].notna()
)

# Initialize u_per with NaN
features_overall_weekly_df['u_per'] = np.nan

# Calculate only for valid rows
lg_AST = features_overall_weekly_df['assists'].mean()
lg_FG = features_overall_weekly_df['fieldGoalsMade'].mean()
lg_FT = features_overall_weekly_df['freeThrowsMade'].mean()
lg_PF = features_overall_weekly_df['foulsPersonal'].mean()
lg_FTA = features_overall_weekly_df['freeThrowsAttempted'].mean()

factor = (2/3) - (0.5 * (lg_AST / lg_FG)) / (2 * (lg_FG / lg_FT))
vop = features_overall_weekly_df['points'].mean() / (features_overall_weekly_df['fieldGoalsAttempted'].mean() - features_overall_weekly_df['reboundsOffensive'].mean() + features_overall_weekly_df['turnovers'].mean() + 0.44 * lg_FTA)
drb_pct = (features_overall_weekly_df['reboundsTotal'].mean() - features_overall_weekly_df['reboundsOffensive'].mean()) / features_overall_weekly_df['reboundsTotal'].mean()

features_overall_weekly_df.loc[valid_mask, 'u_per'] = (1 / features_overall_weekly_df.loc[valid_mask, 'numMinutes']) * (
    features_overall_weekly_df.loc[valid_mask, 'threePointersMade']
    + (2/3) * features_overall_weekly_df.loc[valid_mask, 'assists']
    + (2 - factor * (features_overall_weekly_df.loc[valid_mask, 'team_assists'] / features_overall_weekly_df.loc[valid_mask, 'team_fg'])) * features_overall_weekly_df.loc[valid_mask, 'fieldGoalsMade']
    + (features_overall_weekly_df.loc[valid_mask, 'freeThrowsMade'] * 0.5 * (1 + (1 - (features_overall_weekly_df.loc[valid_mask, 'team_assists'] / features_overall_weekly_df.loc[valid_mask, 'team_fg'])) + (2/3) * (features_overall_weekly_df.loc[valid_mask, 'team_assists'] / features_overall_weekly_df.loc[valid_mask, 'team_fg'])))
    - vop * features_overall_weekly_df.loc[valid_mask, 'turnovers']
    - vop * drb_pct * (features_overall_weekly_df.loc[valid_mask, 'fieldGoalsAttempted'] - features_overall_weekly_df.loc[valid_mask, 'fieldGoalsMade'])
    - vop * 0.44 * (0.44 + (0.56 * drb_pct)) * (features_overall_weekly_df.loc[valid_mask, 'freeThrowsAttempted'] - features_overall_weekly_df.loc[valid_mask, 'freeThrowsMade'])
    + vop * (1 - drb_pct) * (features_overall_weekly_df.loc[valid_mask, 'reboundsTotal'] - features_overall_weekly_df.loc[valid_mask, 'reboundsOffensive'])
    + vop * drb_pct * features_overall_weekly_df.loc[valid_mask, 'reboundsOffensive']
    + vop * features_overall_weekly_df.loc[valid_mask, 'steals']
    + vop * drb_pct * features_overall_weekly_df.loc[valid_mask, 'blocks']
    - features_overall_weekly_df.loc[valid_mask, 'foulsPersonal'] * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * vop)
)

# # Check results
# print("\nAfter filtering:")
# print("inf values:", np.isinf(features_overall_weekly_df['u_per']).sum())
# print("NaN values:", features_overall_weekly_df['u_per'].isna().sum())
# print("Valid u_per values:", valid_mask.sum())

features_overall_weekly_df['pace_adjustment'] = 2 * features_overall_weekly_df['lg_ppg'] / (features_overall_weekly_df['team_ppg_fixed'] + features_overall_weekly_df['avg_opp_score'])
features_overall_weekly_df['a_per'] = features_overall_weekly_df['pace_adjustment'] * features_overall_weekly_df['u_per']
league_avg_aPER = features_overall_weekly_df['a_per'].mean()
features_overall_weekly_df['player_efficiency_rating'] = features_overall_weekly_df['a_per'] * (15 / league_avg_aPER)

# Check final results
print("Final PER distribution:")
print(features_overall_weekly_df['player_efficiency_rating'].describe())
print()
print(f"League average PER: {features_overall_weekly_df['player_efficiency_rating'].mean():.2f} (should be ~15.0)")

# Apply minimum minutes threshold (common in PER calculations)
min_minutes = 5  # or whatever threshold makes sense for your data

features_overall_weekly_df.loc[
    features_overall_weekly_df['numMinutes'] < min_minutes,
    'player_efficiency_rating'
] = np.nan

print(f"\nAfter {min_minutes} minute filter:")
print(features_overall_weekly_df['player_efficiency_rating'].describe())

features_overall_weekly_df = features_overall_weekly_df.drop(columns=['team_assists', 'team_fg','team_ppg_fixed', 'lg_ppg', 'u_per', 'pace_adjustment', 'a_per'])


Minutes == 0: 1250
team_fg == 0: 2
NaN values in key columns:
numMinutes      0
team_assists    0
team_fg         0
dtype: int64
Final PER distribution:
count    234017.000000
mean         15.000000
std          13.340741
min        -199.909466
25%           9.154771
50%          15.167629
75%          21.095042
max         684.869471
Name: player_efficiency_rating, dtype: float64

League average PER: 15.00 (should be ~15.0)

After 5 minute filter:
count    225023.000000
mean         15.208657
std          10.208857
min         -92.017766
25%           9.571464
50%          15.306903
75%          21.048558
max         160.646608
Name: player_efficiency_rating, dtype: float64


In [ ]:
# query = """
# WITH CTE AS (
# SELECT *,
# AVG(player_efficiency_rating) OVER(PARTITION BY team, season, week) as team_avg_PER

# FROM features_overall_weekly_df
# )

# SELECT * FROM CTE

# """

# features_overall_weekly_df = duckdb.query(query).df()
# features_overall_weekly_df

In [ ]:
# Only train on 2021+ seasons to account for modern scoring patterns.
#features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['season'] >= 2021]
#features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-12-22']
# features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-12-15']
# features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-12-08']
# features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-12-01']
# features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-11-24']
# features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-11-17']
# features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-11-10']

In [ ]:
features_overall_weekly_df.columns

Index(['player_id', 'full_name', 'team', 'season', 'week', 'week_start',
       'conference', 'pow_conference', 'games_played_this_week', 'numMinutes',
       'points', 'assists', 'blocks', 'steals', 'reboundsTotal',
       'reboundsDefensive', 'reboundsOffensive', 'fieldGoalsAttempted',
       'fieldGoalsMade', 'threePointersAttempted', 'threePointersMade',
       'freeThrowsAttempted', 'freeThrowsMade', 'turnovers', 'foulsPersonal',
       'plusMinusPoints', 'wins_this_week', 'wins_vs_team_with_all_nba_player',
       'is_win_vs_over_500', 'opponent_has_all_nba', 'avg_opp_score',
       'avg_opp_winrate_prior', 'avg_opp_wins_prior', 'avg_opp_losses_prior',
       'away_games_prior', 'away_losses_prior', 'away_win_streak_prior',
       'away_wins_prior', 'home_games_prior', 'home_losses_prior',
       'home_win_streak_prior', 'home_wins_prior', 'losses_prior',
       'wins_vs_over_500_prior', 'won_player_of_the_week',
       'all_star_this_season', 'mvp_this_season',
       'all_nba_f

In [ ]:
# View the class imbalance (99.4% positive class, 0.6% negative class)
print(features_overall_weekly_df['won_player_of_the_week'].value_counts())
print(features_overall_weekly_df['won_player_of_the_week'].value_counts(normalize=True))

won_player_of_the_week
0    233872
1      1397
Name: count, dtype: int64
won_player_of_the_week
0    0.994062
1    0.005938
Name: proportion, dtype: float64


In [ ]:
# Drop columns that contain the answer. Also drop mvp, all-star, all-nba because they are from the future when we try to do inference during season
leakage_cols = ['pow_player_id', 'player_of_the_week', 'pow_conference',
                'all_star_this_season', 'mvp_this_season',
                'all_nba_first_team_this_season', 'all_nba_second_team_this_season',
                'all_nba_third_team_this_season']

# Set aside identifier columns
id_cols = ['player_id', 'full_name', 'team', 'season', 'week', 'week_start', 'conference']

features_overall_weekly_df_clean = features_overall_weekly_df.drop(columns=leakage_cols + id_cols)
df_encoded = pd.get_dummies(features_overall_weekly_df_clean, drop_first=True)
df_encoded

,games_played_this_week,numMinutes,points,assists,blocks,steals,reboundsTotal,reboundsDefensive,reboundsOffensive,fieldGoalsAttempted,...,points_std_season,assists_mean_season,assists_std_season,plusMinusPoints_mean_season,plusMinusPoints_std_season,z_pts,z_ast,z_pm,breakout_score,player_efficiency_rating
0,4,142.00,83.0,11.0,11.0,2.0,43.0,32.0,11.0,65.0,...,12.666667,6.666667,3.559026,0.000000,0.000000,2.157895,1.217562,0.000000,1.444216,22.120472
1,3,84.00,50.0,17.0,0.0,1.0,13.0,2.0,1.0,36.0,...,29.111853,16.250000,7.822244,0.000000,0.000000,-0.841582,0.095880,0.000000,-0.392027,20.341994
2,3,106.00,83.0,19.0,16.0,6.0,21.0,14.0,7.0,51.0,...,25.521233,17.444444,6.075655,0.000000,0.000000,-0.444075,0.256031,0.000000,-0.145228,40.606844
3,3,23.00,0.0,3.0,1.0,0.0,4.0,4.0,0.0,2.0,...,13.917166,4.000000,2.915476,0.000000,0.000000,-1.095769,-0.342997,0.000000,-0.650784,2.127043
4,3,92.00,53.0,10.0,7.0,4.0,28.0,4.0,2.0,40.0,...,23.060307,13.777778,6.778688,0.000000,0.000000,0.404736,-0.557302,0.000000,0.035177,24.934914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235264,1,0.25,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.947457,1.000000,0.707107,-5.625000,9.745993,-0.424094,-1.414214,0.474554,-0.541400,NaN
235265,1,8.46,4.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,...,18.751568,1.538462,1.946017,-8.461538,13.793061,-0.713786,-0.790569,0.613463,-0.471371,16.732762
235266,3,83.90,33.0,4.0,0.0,3.0,6.0,6.0,0.0,34.0,...,10.443238,2.357143,1.797107,-10.214286,14.173308,1.600489,0.914168,-2.524867,0.569521,9.509411
235267,3,53.76,17.0,1.0,2.0,1.0,5.0,1.0,4.0,14.0,...,1.160577,0.142857,0.349927,-2.571429,5.948761,14.032430,2.449490,-0.072044,7.736653,10.336200


In [ ]:
from sklearn.model_selection import GroupShuffleSplit

X = df_encoded.drop(columns=['won_player_of_the_week'])
y = df_encoded['won_player_of_the_week']
identifiers = features_overall_weekly_df[id_cols]

# Create groups for each season-week-conference combination
groups = (features_overall_weekly_df['season'].astype(str) + '_' +
          features_overall_weekly_df['week'].astype(str) + '_' +
          features_overall_weekly_df['conference'].astype(str))

# Use GroupShuffleSplit instead of train_test_split
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

# Split using the indices
X_train = X.iloc[train_idx]
X_test = X.iloc[test_idx]
y_train = y.iloc[train_idx]
y_test = y.iloc[test_idx]
id_train = identifiers.iloc[train_idx]
id_test = identifiers.iloc[test_idx]

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Positive class in y_train: {100 * round(y_train.mean(),4)}%")
print(f"Positive class in y_test: {100 * round(y_test.mean(),4)}%")

Training samples: 187552
Test samples: 47717
Positive class in y_train: 0.6%
Positive class in y_test: 0.59%


In [ ]:
# Train LightGBM model
lgb_model = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    num_leaves=31,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

# Train LightGBM model
lgb_model.fit(X_train, y_train)

# Get predictions
y_pred_proba = lgb_model.predict_proba(X_test)[:, 1]

In [ ]:
evaluation_df = id_test.copy()
evaluation_df['actual_won'] = y_test.values
evaluation_df['pred_proba'] = y_pred_proba
evaluation_df

,player_id,full_name,team,season,week,week_start,conference,actual_won,pred_proba
1455,3,Grant Long,Heat,1988,10,1989-03-06,East,0,0.000757
1456,22,Rik Smits,Pacers,1988,10,1989-03-06,East,0,0.003224
1457,23,Dennis Rodman,Pistons,1988,10,1989-03-06,East,0,0.001355
1458,52,Micheal Williams,Pistons,1988,10,1989-03-06,East,0,0.000085
1459,73,Hot Rod Williams,Cavaliers,1988,10,1989-03-06,East,0,0.001912
...,...,...,...,...,...,...,...,...,...
235262,1642382,Branden Carlson,Thunder,2024,12,2025-03-17,West,0,0.000063
235263,1642384,Isaiah Crawford,Kings,2024,12,2025-03-17,West,0,0.000124
235265,1642403,Isaac Jones,Kings,2024,12,2025-03-17,West,0,0.000116
235267,1642461,Spencer Jones,Nuggets,2024,12,2025-03-17,West,0,0.000035


In [ ]:
actual_winners_df = evaluation_df[evaluation_df['actual_won'] == 1]
actual_winners_df.head()

,player_id,full_name,team,season,week,week_start,conference,actual_won,pred_proba
1593,78318,Isiah Thomas,Pistons,1988,10,1989-03-06,East,1,0.975065
3529,686,Antonio McDyess,Nuggets,1998,8,1999-02-22,West,1,0.972441
3923,406,Shaquille O'Neal,Lakers,1999,8,2000-02-21,West,1,0.988290
4252,1495,Tim Duncan,Spurs,1999,13,2000-03-27,West,1,0.984656
5414,947,Allen Iverson,76ers,2002,48,2002-11-25,East,1,0.979060


In [ ]:
print(f"Total samples in evaluation_df: {len(evaluation_df)}")
print(f"Actual winners: {len(actual_winners_df)}")

Total samples in evaluation_df: 47717
Actual winners: 280


In [ ]:
# How many season,conference,week combinations were there in actual_winners_df? This is our denominator for accuracy.

query = """
SELECT COUNT(DISTINCT(season,conference,week)) AS total_season_conference_weeks_we_couldve_predicted
FROM actual_winners_df
"""

total_season_conference_weeks_we_couldve_predicted = duckdb.query(query).df()['total_season_conference_weeks_we_couldve_predicted'].sum()
total_season_conference_weeks_we_couldve_predicted

np.int64(277)

In [ ]:
# How many of the actual winners did we predict?
# Aka how many of the actual winners did our model assign a top-k probability for that [season,conference,week] combo?
for k in [1, 2, 3, 4, 5, 10]:
  query = f"""

  WITH Highest_Probabilities AS (
  SELECT *
  ,RANK() OVER(PARTITION BY season, conference, week ORDER BY pred_proba DESC) AS rnk
  FROM evaluation_df
  )
  ,
  Actual_Winners AS (
    SELECT * FROM evaluation_df
    WHERE actual_won = 1
  )


  SELECT * FROM Highest_Probabilities
  JOIN Actual_Winners
  ON
  Highest_Probabilities.season = Actual_Winners.season
  AND
  Highest_Probabilities.week = Actual_Winners.week
  AND
  Highest_Probabilities.conference = Actual_Winners.conference
  WHERE rnk <= {k}
  AND Highest_Probabilities.pred_proba = Actual_Winners.pred_proba
  """
  duckdb.query(query).df()
  correct_predictions_df = duckdb.query(query).df()
  total_correct_predictions = correct_predictions_df["actual_won"].sum()
  print(f"Top-{k} accuracy:", round(100*total_correct_predictions/total_season_conference_weeks_we_couldve_predicted,2),"%")

Top-1 accuracy: 56.32 %
Top-2 accuracy: 73.65 %
Top-3 accuracy: 81.59 %
Top-4 accuracy: 86.28 %
Top-5 accuracy: 89.89 %
Top-10 accuracy: 97.11 %


In [ ]:
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': lgb_model.feature_importances_
}).sort_values('importance', ascending=False)

feature_importance['importance_pct'] = (feature_importance['importance'] / feature_importance['importance'].sum() * 100)

print("LightGBM Top 20 features:")
print(feature_importance.head(20)[['feature','importance_pct']])

LightGBM Top 20 features:
                     feature  importance_pct
54  player_efficiency_rating        8.517034
36                  team_pts        5.711423
2                     points        5.010020
18            wins_this_week        4.342017
44        points_mean_season        3.540414
1                 numMinutes        2.939212
39                  team_stl        2.839011
10            fieldGoalsMade        2.705411
46       assists_mean_season        2.371409
17           plusMinusPoints        2.304609
15                 turnovers        2.304609
9        fieldGoalsAttempted        2.237809
6              reboundsTotal        2.237809
53            breakout_score        2.171009
41      fieldGoalsPercentage        2.137609
3                    assists        2.037408
13       freeThrowsAttempted        2.037408
37                  team_ast        2.004008
38                  team_blk        2.004008
22             avg_opp_score        1.937208


In [ ]:
# import joblib

# # Save model
# joblib.dump(lgb_model, 'lightgbm_potw_model.pkl')

# # Save feature names
# joblib.dump(X_train.columns.tolist(), 'lightgbm_model_features.pkl')

In [ ]:
print("\nTraining final model on 100% of data for deployment...")
final_model = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    num_leaves=31,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

final_model.fit(X, y)  # ALL data

# 5. Save the final model
joblib.dump(final_model, 'lightgbm_potw_model.pkl')
joblib.dump(X.columns.tolist(), 'lightgbm_model_features.pkl')


Training final model on 100% of data for deployment...


['lightgbm_model_features.pkl']

In [ ]:
# Feature importance from final model
final_feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

final_feature_importance['importance_pct'] = (final_feature_importance['importance'] / final_feature_importance['importance'].sum() * 100)

print("Final Model Top 20 features:")
print(final_feature_importance.head(20)[['feature','importance_pct']])

Final Model Top 20 features:
                       feature  importance_pct
54    player_efficiency_rating        9.103034
36                    team_pts        6.535512
2                       points        5.468489
18              wins_this_week        5.135045
1                   numMinutes        3.501167
44          points_mean_season        3.401134
10              fieldGoalsMade        2.500834
39                    team_stl        2.467489
50                       z_pts        2.300767
7            reboundsDefensive        2.267422
37                    team_ast        2.200734
3                      assists        2.100700
53              breakout_score        2.100700
17             plusMinusPoints        2.034011
9          fieldGoalsAttempted        2.034011
46         assists_mean_season        2.000667
13         freeThrowsAttempted        2.000667
15                   turnovers        1.967322
6                reboundsTotal        1.967322
49  plusMinusPoints_std_season 

**The below steps will use the above model for inference.**

In [ ]:
def get_potw_predictions(week_start: str):
  """
  get_potw_predictions() does inference to predict POW.
  week_start must be a Monday in 'yyyy-mm-dd' format
  """
  import duckdb
  import joblib
  import wget

  # features-overall-weekly-for-inference.csv is calculated in a separate script that runs within the daily production pipeline
  # features-overall-weekly-for-inference.csv is needed to make predictions while the current week is in-progress (i.e. no POW announced yet)
  wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/features-overall-weekly-for-inference.csv')

  # Read in model & its feature names
  model = joblib.load('lightgbm_potw_model.pkl')
  feature_names = joblib.load('lightgbm_model_features.pkl')

  # Read in player-week stats
  overall_weekly_agg_df = pd.read_csv('features-overall-weekly-for-inference.csv')

  # Remove unwanted features
  overall_weekly_agg_df = overall_weekly_agg_df.drop(columns=['league_pts_mean', 'league_pts_std',
       'league_ast_mean', 'league_ast_std', 'league_pm_mean', 'league_pm_std',
       'z_s_pts', 'z_s_ast', 'z_s_pm'])


##############################################################
##############################################################

  query = """
  WITH CTE AS (
  SELECT team, season, week, SUM(assists) AS team_assists
  FROM overall_weekly_agg_df
  GROUP BY team, season, week
  )
  ,CTE2 AS (

  SELECT team, season, week, SUM(fieldGoalsMade) AS team_fg
  FROM overall_weekly_agg_df
  GROUP BY team, season, week
  )
  , CTE3 AS (
  SELECT CTE.team, CTE.season, CTE.week, team_assists, team_fg
  FROM CTE
  JOIN CTE2
  ON CTE.team = CTE2.team AND CTE.season = CTE2.season AND CTE.week = CTE2.week
  )
  , CTE4 AS (

  SELECT team, season, week, SUM(points)/team_gms AS team_ppg
  FROM overall_weekly_agg_df
  GROUP BY team, season, week, team_gms
  )

  , CTE5 AS (
  SELECT *
  ,team_pts/team_gms AS team_ppg
  FROM overall_weekly_agg_df a
  JOIN CTE3 b
  ON a.team=b.team AND a.season = b.season AND a.week=b.week
  )

  , CTE6 AS (
    SELECT *,
    CASE WHEN team_ppg > 186 THEN 115 ELSE team_ppg END AS team_ppg_fixed  -- If team_ppg shows as greater than 186, set it to 115
    FROM CTE5
  )

  SELECT *,
  AVG(team_ppg_fixed) OVER(PARTITION BY season, week) AS lg_ppg
  FROM CTE6

  """
  overall_weekly_agg_df = duckdb.query(query).df().drop(columns=['team_1','season_1','week_1','team_ppg'])
  overall_weekly_agg_df

  # League-wide statistics (scalars)
  lg_AST = overall_weekly_agg_df['assists'].mean()
  lg_FG = overall_weekly_agg_df['fieldGoalsMade'].mean()
  lg_FT = overall_weekly_agg_df['freeThrowsMade'].mean()
  lg_PF = overall_weekly_agg_df['foulsPersonal'].mean()
  lg_FTA = overall_weekly_agg_df['freeThrowsAttempted'].mean()

  factor = (2/3) - (0.5 * (lg_AST / lg_FG)) / (2 * (lg_FG / lg_FT))
  vop = overall_weekly_agg_df['points'].mean() / (overall_weekly_agg_df['fieldGoalsAttempted'].mean() - overall_weekly_agg_df['reboundsOffensive'].mean() + overall_weekly_agg_df['turnovers'].mean() + 0.44 * lg_FTA)
  drb_pct = (overall_weekly_agg_df['reboundsTotal'].mean() - overall_weekly_agg_df['reboundsOffensive'].mean()) / overall_weekly_agg_df['reboundsTotal'].mean()

  overall_weekly_agg_df['u_per'] = (1 / overall_weekly_agg_df['numMinutes']) * (
      overall_weekly_agg_df['threePointersMade']
      + (2/3) * overall_weekly_agg_df['assists']
      + (2 - factor * (overall_weekly_agg_df['team_assists'] / overall_weekly_agg_df['team_fg'])) * overall_weekly_agg_df['fieldGoalsMade']
      + (overall_weekly_agg_df['freeThrowsMade'] * 0.5 * (1 + (1 - (overall_weekly_agg_df['team_assists'] / overall_weekly_agg_df['team_fg'])) + (2/3) * (overall_weekly_agg_df['team_assists'] / overall_weekly_agg_df['team_fg'])))
      - vop * overall_weekly_agg_df['turnovers']
      - vop * drb_pct * (overall_weekly_agg_df['fieldGoalsAttempted'] - overall_weekly_agg_df['fieldGoalsMade'])
      - vop * 0.44 * (0.44 + (0.56 * drb_pct)) * (overall_weekly_agg_df['freeThrowsAttempted'] - overall_weekly_agg_df['freeThrowsMade'])
      + vop * (1 - drb_pct) * (overall_weekly_agg_df['reboundsTotal'] - overall_weekly_agg_df['reboundsOffensive'])
      + vop * drb_pct * overall_weekly_agg_df['reboundsOffensive']
      + vop * overall_weekly_agg_df['steals']
      + vop * drb_pct * overall_weekly_agg_df['blocks']
      - overall_weekly_agg_df['foulsPersonal'] * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * vop)
  )


  # Filter to only calculate for valid rows
  valid_mask = (
      (overall_weekly_agg_df['numMinutes'] > 0) &
      (overall_weekly_agg_df['team_fg'] > 0) &
      overall_weekly_agg_df['numMinutes'].notna()
  )

  # Initialize u_per with NaN
  overall_weekly_agg_df['u_per'] = np.nan

  # Calculate only for valid rows
  lg_AST = overall_weekly_agg_df['assists'].mean()
  lg_FG = overall_weekly_agg_df['fieldGoalsMade'].mean()
  lg_FT = overall_weekly_agg_df['freeThrowsMade'].mean()
  lg_PF = overall_weekly_agg_df['foulsPersonal'].mean()
  lg_FTA = overall_weekly_agg_df['freeThrowsAttempted'].mean()

  factor = (2/3) - (0.5 * (lg_AST / lg_FG)) / (2 * (lg_FG / lg_FT))
  vop = overall_weekly_agg_df['points'].mean() / (overall_weekly_agg_df['fieldGoalsAttempted'].mean() - overall_weekly_agg_df['reboundsOffensive'].mean() + overall_weekly_agg_df['turnovers'].mean() + 0.44 * lg_FTA)
  drb_pct = (overall_weekly_agg_df['reboundsTotal'].mean() - overall_weekly_agg_df['reboundsOffensive'].mean()) / overall_weekly_agg_df['reboundsTotal'].mean()

  overall_weekly_agg_df.loc[valid_mask, 'u_per'] = (1 / overall_weekly_agg_df.loc[valid_mask, 'numMinutes']) * (
      overall_weekly_agg_df.loc[valid_mask, 'threePointersMade']
      + (2/3) * overall_weekly_agg_df.loc[valid_mask, 'assists']
      + (2 - factor * (overall_weekly_agg_df.loc[valid_mask, 'team_assists'] / overall_weekly_agg_df.loc[valid_mask, 'team_fg'])) * overall_weekly_agg_df.loc[valid_mask, 'fieldGoalsMade']
      + (overall_weekly_agg_df.loc[valid_mask, 'freeThrowsMade'] * 0.5 * (1 + (1 - (overall_weekly_agg_df.loc[valid_mask, 'team_assists'] / overall_weekly_agg_df.loc[valid_mask, 'team_fg'])) + (2/3) * (overall_weekly_agg_df.loc[valid_mask, 'team_assists'] / overall_weekly_agg_df.loc[valid_mask, 'team_fg'])))
      - vop * overall_weekly_agg_df.loc[valid_mask, 'turnovers']
      - vop * drb_pct * (overall_weekly_agg_df.loc[valid_mask, 'fieldGoalsAttempted'] - overall_weekly_agg_df.loc[valid_mask, 'fieldGoalsMade'])
      - vop * 0.44 * (0.44 + (0.56 * drb_pct)) * (overall_weekly_agg_df.loc[valid_mask, 'freeThrowsAttempted'] - overall_weekly_agg_df.loc[valid_mask, 'freeThrowsMade'])
      + vop * (1 - drb_pct) * (overall_weekly_agg_df.loc[valid_mask, 'reboundsTotal'] - overall_weekly_agg_df.loc[valid_mask, 'reboundsOffensive'])
      + vop * drb_pct * overall_weekly_agg_df.loc[valid_mask, 'reboundsOffensive']
      + vop * overall_weekly_agg_df.loc[valid_mask, 'steals']
      + vop * drb_pct * overall_weekly_agg_df.loc[valid_mask, 'blocks']
      - overall_weekly_agg_df.loc[valid_mask, 'foulsPersonal'] * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * vop)
  )


  overall_weekly_agg_df['pace_adjustment'] = 2 * overall_weekly_agg_df['lg_ppg'] / (overall_weekly_agg_df['team_ppg_fixed'] + overall_weekly_agg_df['avg_opp_score'])
  overall_weekly_agg_df['a_per'] = overall_weekly_agg_df['pace_adjustment'] * overall_weekly_agg_df['u_per']
  league_avg_aPER = overall_weekly_agg_df['a_per'].mean()
  overall_weekly_agg_df['player_efficiency_rating'] = overall_weekly_agg_df['a_per'] * (15 / league_avg_aPER)


  # Apply minimum minutes threshold (common in PER calculations)
  min_minutes = 5  # or whatever threshold makes sense for your data

  overall_weekly_agg_df.loc[
      overall_weekly_agg_df['numMinutes'] < min_minutes,
      'player_efficiency_rating'
  ] = np.nan


  overall_weekly_agg_df = overall_weekly_agg_df.drop(columns=['team_assists', 'team_fg','team_ppg_fixed', 'lg_ppg', 'u_per', 'pace_adjustment', 'a_per'])


################################################################
################################################################

  # Query all players who played for the input week
  query = f"""
  SELECT * FROM overall_weekly_agg_df
  WHERE week_start = '{week_start}'
  """
  df = duckdb.query(query).df()

  #Keep player info separate
  player_info = df[['player_id', 'full_name', 'conference', 'season', 'week']]
  X = df[feature_names]
  probabilities = model.predict_proba(X)[:, 1]

  # Attach probabilities to the player identities
  results = player_info.copy()
  results['probability'] = probabilities

  # Now partition by season, week, and conference and return top 5 for each conference
  results
  query = f"""
  WITH CTE AS (SELECT *,
  RANK() OVER(PARTITION BY season, week, conference ORDER BY probability DESC) AS rank
  FROM results
  )

  ,CTE2 AS (
  SELECT * FROM CTE WHERE conference = 'East' AND rank <= 5
  UNION ALL
  SELECT * FROM CTE WHERE conference = 'West' AND rank <= 5

  )
  SELECT
  '{week_start}' AS week_start
  ,rank
  ,full_name as name
  ,conference
  ,probability
  FROM CTE2
  """

  top_five_per_conference = duckdb.query(query).df()

  return top_five_per_conference

get_potw_predictions('2025-12-22')

,week_start,rank,name,conference,probability
0,2025-12-22,1,Karl-Anthony Towns,East,0.656866
1,2025-12-22,2,Kevin Porter Jr.,East,0.520688
2,2025-12-22,3,Jaylen Brown,East,0.466286
3,2025-12-22,4,Scottie Barnes,East,0.392603
4,2025-12-22,5,Nic Claxton,East,0.245099
5,2025-12-22,1,Kawhi Leonard,West,0.603993
6,2025-12-22,2,Stephon Castle,West,0.585991
7,2025-12-22,3,Amen Thompson,West,0.482635
8,2025-12-22,4,LeBron James,West,0.451301
9,2025-12-22,5,Julius Randle,West,0.380631


AutoGluon Model

In [ ]:
# def get_potw_predictions(week_start: str):
#   """
#   get_potw_predictions() does inference to predict POW.
#   week_start must be a Monday in 'yyyy-mm-dd' format
#   """
#   import duckdb
#   import joblib
#   import wget
#   import pandas as pd
#   from autogluon.tabular import TabularPredictor

#   # features-overall-weekly-for-inference.csv is calculated in a separate script that runs within the daily production pipeline
#   # features-overall-weekly-for-inference.csv is needed to make predictions while the current week is in-progress (i.e. no POW announced yet)
#   wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/features-overall-weekly-for-inference.csv')
#   wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/autogluon_model_features.pkl')

#   # Read in model (load the full predictor, not just the pkl)
#   predictor = TabularPredictor.load('./ag_models')
#   feature_names = joblib.load('autogluon_model_features.pkl')

#   # Read in player-week stats
#   overall_weekly_agg_df = pd.read_csv('features-overall-weekly-for-inference.csv')

#   # Remove unwanted features
#   overall_weekly_agg_df = overall_weekly_agg_df.drop(columns=['league_pts_mean', 'league_pts_std',
#        'league_ast_mean', 'league_ast_std', 'league_pm_mean', 'league_pm_std',
#        'z_s_pts', 'z_s_ast', 'z_s_pm'])

#   # Query all players who played for the input week
#   query = f"""
#   SELECT * FROM overall_weekly_agg_df
#   WHERE week_start = '{week_start}'
#   """
#   df = duckdb.query(query).df()

#   #Keep player info separate
#   player_info = df[['player_id', 'full_name', 'conference', 'season', 'week']]
#   X = df[feature_names]

#   # Use predictor.predict_proba instead
#   probabilities = predictor.predict_proba(X)[1].values  # Get probability of class 1

#   # Attach probabilities to the player identities
#   results = player_info.copy()
#   results['probability'] = probabilities

#   # Now partition by season, week, and conference and return top 5 for each conference
#   query = f"""
#   WITH CTE AS (SELECT *,
#   RANK() OVER(PARTITION BY season, week, conference ORDER BY probability DESC) AS rank
#   FROM results
#   )

#   ,CTE2 AS (
#   SELECT * FROM CTE WHERE conference = 'East' AND rank <= 5
#   UNION ALL
#   SELECT * FROM CTE WHERE conference = 'West' AND rank <= 5

#   )
#   SELECT
#   '{week_start}' AS week_start
#   ,rank
#   ,full_name as name
#   ,conference
#   ,probability
#   FROM CTE2
#   """

#   top_five_per_conference = duckdb.query(query).df()

#   return top_five_per_conference

# get_potw_predictions('2025-12-15')

Pick out a single model from autogluon and see results e.g. "LightGBM_BAG_L1"

In [ ]:
# def get_potw_predictions(week_start: str):
#   """
#   get_potw_predictions() does inference to predict POW.
#   week_start must be a Monday in 'yyyy-mm-dd' format
#   """
#   import duckdb
#   import joblib
#   import wget
#   import pandas as pd
#   from autogluon.tabular import TabularPredictor

#   # features-overall-weekly-for-inference.csv is calculated in a separate script that runs within the daily production pipeline
#   # features-overall-weekly-for-inference.csv is needed to make predictions while the current week is in-progress (i.e. no POW announced yet)
#   #wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/features-overall-weekly-for-inference.csv')
#   #wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/autogluon_model_features.pkl')

#   # Read in model (load the full predictor, not just the pkl)
#   predictor = TabularPredictor.load('./ag_models')
#   feature_names = joblib.load('autogluon_model_features.pkl')

#   # Read in player-week stats
#   overall_weekly_agg_df = pd.read_csv('features-overall-weekly-for-inference.csv')

#   # Remove unwanted features
#   overall_weekly_agg_df = overall_weekly_agg_df.drop(columns=['league_pts_mean', 'league_pts_std',
#        'league_ast_mean', 'league_ast_std', 'league_pm_mean', 'league_pm_std',
#        'z_s_pts', 'z_s_ast', 'z_s_pm'])

#   # Query all players who played for the input week
#   query = f"""
#   SELECT * FROM overall_weekly_agg_df
#   WHERE week_start = '{week_start}'
#   """
#   df = duckdb.query(query).df()

#   #Keep player info separate
#   player_info = df[['player_id', 'full_name', 'conference', 'season', 'week']]
#   X = df[feature_names]

#   # Use predictor.predict_proba with model='LightGBM_BAG_L1'
#   probabilities = predictor.predict_proba(X, model='LightGBM_BAG_L1')[1].values  # Get probability of class 1

#   # Attach probabilities to the player identities
#   results = player_info.copy()
#   results['probability'] = probabilities

#   # Now partition by season, week, and conference and return top 5 for each conference
#   query = f"""
#   WITH CTE AS (SELECT *,
#   RANK() OVER(PARTITION BY season, week, conference ORDER BY probability DESC) AS rank
#   FROM results
#   )

#   ,CTE2 AS (
#   SELECT * FROM CTE WHERE conference = 'East' AND rank <= 5
#   UNION ALL
#   SELECT * FROM CTE WHERE conference = 'West' AND rank <= 5

#   )
#   SELECT
#   '{week_start}' AS week_start
#   ,rank
#   ,full_name as name
#   ,conference
#   ,probability
#   FROM CTE2
#   """

#   top_five_per_conference = duckdb.query(query).df()

#   return top_five_per_conference

# get_potw_predictions('2025-12-15')

In [2]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=90cb167f9749c3c8f1631078aab723458dab60110f500b1237da3eec9ec6f5b7
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [5]:
import wget
import pandas as pd
import duckdb

In [23]:
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/features-overall.csv')

In [65]:
features_overall_df = pd.read_csv('features-overall.csv')

In [66]:
features_overall_df.columns

Index(['gameId', 'gameDate', 'day', 'week', 'month', 'year', 'team', 'teamid',
       'firstName', 'lastName', 'full_name', 'player_id', 'opponent',
       'opponentid', 'outcome', 'home', 'team_score', 'opp_score',
       'games_prior', 'wins_prior', 'losses_prior', 'record_prior',
       'home_games_prior', 'home_wins_prior', 'home_losses_prior',
       'home_record_prior', 'away_games_prior', 'away_wins_prior',
       'away_losses_prior', 'away_record_prior', 'win_streak_prior',
       'home_win_streak_prior', 'away_win_streak_prior', 'opp_wins_prior',
       'opp_losses_prior', 'opp_winrate_prior', 'is_win_vs_over_500',
       'wins_vs_over_500_prior', 'week_games_prior', 'week_wins_prior',
       'week_losses_prior', 'week_record_prior', 'season', 'pow_player_id',
       'player_of_the_week', 'pow_conference', 'pow_last_date_of_week',
       'numMinutes', 'points', 'assists', 'blocks', 'steals',
       'fieldGoalsAttempted', 'fieldGoalsMade', 'fieldGoalsPercentage',
       'threeP

In [67]:
features_overall_df = features_overall_df[['gameId', 'gameDate', 'team',
'full_name',
        'opponent',

       'home', 'team_score', 'opp_score',

       'points',
       'fieldGoalsAttempted', 'fieldGoalsMade',

       'win']]
features_overall_df

,gameId,gameDate,team,full_name,opponent,home,team_score,opp_score,points,fieldGoalsAttempted,fieldGoalsMade,win
0,21100529,2012-03-02,Celtics,Kevin Garnett,Nets,1,107,94,20,13,10,1
1,21100530,2012-03-02,Hawks,Jerry Stackhouse,Bucks,1,99,94,8,7,2,1
2,21100528,2012-03-02,Raptors,Linas Kleiza,Grizzlies,1,99,102,12,8,4,0
3,21100528,2012-03-02,Raptors,Amir Johnson,Grizzlies,1,99,102,10,7,5,0
4,21100528,2012-03-02,Raptors,Jose Calderon,Grizzlies,1,99,102,10,9,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
936192,29100081,1991-11-12,Hornets,Rex Chapman,Hawks,0,100,118,13,13,6,0
936193,29100081,1991-11-12,Hornets,Kendall Gill,Hawks,0,100,118,9,8,4,0
936194,29100081,1991-11-12,Hornets,J.R. Reid,Hawks,0,100,118,0,3,0,0
936195,29100081,1991-11-12,Hornets,Anthony Frederick,Hawks,0,100,118,12,8,5,0


In [86]:
query = """
WITH combined_teams AS (
SELECT DISTINCT

a.gameId,
a.gameDate,
a.team,
a.full_name,
a.opponent,
a.home,
a.team_score,
a.opp_score,
a.points,
a.fieldGoalsAttempted,
a.fieldGoalsMade,
a.win,


b.full_name as opp_player_full_name,
b.fieldGoalsAttempted AS opp_player_fieldGoalsAttempted,
b.fieldGoalsMade AS opp_player_fieldGoalsMade

FROM
features_overall_df a
JOIN features_overall_df b
ON a.gameId = b.gameId

)
, home_fgs_by_player AS (
SELECT DISTINCT gameId, gameDate, team,full_name,fieldGoalsAttempted
FROM combined_teams
WHERE home = 1
)
, home_fg_aggregate AS (

  SELECT gameId, gameDate, team as home_team, SUM(fieldGoalsAttempted) as home_team_fga
  FROM home_fgs_by_player
  GROUP BY gameId, gameDate, team
)

, away_fgs_by_player AS (
SELECT DISTINCT gameId, gameDate, team,full_name,fieldGoalsAttempted
FROM combined_teams
WHERE home = 0
)
, away_fg_aggregate AS (

  SELECT gameId, gameDate, team as away_team, SUM(fieldGoalsAttempted) as away_team_fga
  FROM away_fgs_by_player
  GROUP BY gameId, gameDate, team
)

, home_away_fg_comparison AS (

SELECT home_fg_aggregate.gameId,
home_fg_aggregate.gameDate,
home_team,
CAST(home_team_fga AS INT) as home_team_fga,
away_team,
CAST(away_team_fga AS INT) as away_team_fga
FROM home_fg_aggregate
JOIN away_fg_aggregate
ON home_fg_aggregate.gameId = away_fg_aggregate.gameId

)
, combine_fg_aggregate_with_base_table AS (
SELECT *
FROM home_away_fg_comparison
JOIN features_overall_df
ON home_away_fg_comparison.gameId = features_overall_df.gameId
)

SELECT DISTINCT

gameId
,CAST(gameDate AS DATE) AS gameDate
,team
,opponent
,CASE
  WHEN team = home_team THEN home_team_fga
  WHEN team = away_team THEN away_team_fga
  ELSE NULL
END AS team_fga
,CASE
  WHEN team = home_team THEN away_team_fga
  WHEN team = away_team THEN home_team_fga
  ELSE NULL
END AS opp_team_fga
,team_score AS team_pts
,opp_score AS opp_team_pts
FROM combine_fg_aggregate_with_base_table

"""

field_goal_features_df = duckdb.query(query).df()
field_goal_features_df['week_start'] = field_goal_features_df['gameDate'] - field_goal_features_df['gameDate'].dt.weekday.astype('timedelta64[D]')
field_goal_features_df

query = """

SELECT week_start
,team
,CAST(SUM(team_fga) AS INT) AS team_fga
,CAST(SUM(opp_team_fga) AS INT) AS opp_team_fga
,CAST(SUM(team_pts) AS INT) AS team_pts
,CAST(SUM(opp_team_pts) AS INT) AS opp_team_pts
FROM field_goal_features_df
GROUP BY week_start, team
"""


field_goal_features_weekly_agg_df = duckdb.query(query).df()
field_goal_features_weekly_agg_df


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,week_start,team,team_fga,opp_team_fga,team_pts,opp_team_pts
0,2021-01-18,Cavaliers,281,286,375,389
1,2021-01-18,76ers,242,265,353,329
2,2021-01-18,Raptors,350,317,426,387
3,2021-01-18,Bucks,283,255,358,353
4,2021-01-18,Nets,360,390,501,519
...,...,...,...,...,...,...
20578,1991-12-16,Knicks,67,85,102,94
20579,1991-12-02,Hornets,79,93,124,106
20580,1991-11-25,Timberwolves,147,180,179,171
20581,1991-11-18,Hawks,174,163,214,218


In [82]:
query = """

SELECT week_start
,team
,CAST(SUM(team_fga) AS INT) AS team_fga
,CAST(SUM(opp_team_fga) AS INT) AS opp_team_fga
,CAST(SUM(team_pts) AS INT) AS team_pts
,CAST(SUM(opp_team_pts) AS INT) AS opp_team_pts
FROM field_goal_features_df
GROUP BY week_start, team
"""


duckdb.query(query).df()

,week_start,team,team_fga,opp_team_fga,team_pts,opp_team_pts
0,2012-02-27,Magic,240,243,318,298
1,2012-02-27,Hornets,338,330,360,388
2,2012-02-27,Bobcats,245,231,267,315
3,2012-02-27,Knicks,170,185,231,218
4,2012-02-27,Wizards,264,249,314,319
...,...,...,...,...,...,...
20578,1993-02-08,Celtics,70,83,104,92
20579,1993-02-01,Nets,81,83,105,102
20580,1993-01-25,Lakers,93,83,123,108
20581,1993-01-25,Timberwolves,165,180,213,219


In [83]:
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/features-overall-weekly.csv')
features_overall_weekly_df = pd.read_csv('features-overall-weekly.csv')

In [94]:
features_overall_weekly_df.shape

(235269, 79)

In [92]:
features_overall_weekly_df['week_start'] = pd.to_datetime(features_overall_weekly_df['week_start'])

In [99]:
features_overall_weekly_df.merge(field_goal_features_weekly_agg_df,on=['team','week_start'],how='left').columns

Index(['player_id', 'full_name', 'team', 'season', 'week', 'week_start',
       'conference', 'pow_conference', 'games_played_this_week', 'numMinutes',
       'points', 'assists', 'blocks', 'steals', 'reboundsTotal',
       'reboundsDefensive', 'reboundsOffensive', 'fieldGoalsAttempted',
       'fieldGoalsMade', 'threePointersAttempted', 'threePointersMade',
       'freeThrowsAttempted', 'freeThrowsMade', 'turnovers', 'foulsPersonal',
       'plusMinusPoints', 'wins_this_week', 'wins_vs_team_with_all_nba_player',
       'is_win_vs_over_500', 'opponent_has_all_nba', 'avg_opp_score',
       'avg_opp_winrate_prior', 'avg_opp_wins_prior', 'avg_opp_losses_prior',
       'away_games_prior', 'away_losses_prior', 'away_win_streak_prior',
       'away_wins_prior', 'home_games_prior', 'home_losses_prior',
       'home_win_streak_prior', 'home_wins_prior', 'losses_prior',
       'wins_vs_over_500_prior', 'won_player_of_the_week',
       'all_star_this_season', 'mvp_this_season',
       'all_nba_f